# RePlay recommender models comparison

### Dataset
We will compare RePlay models on __MovieLens 1m__. 

### Dataset preprocessing: 
Ratings greater than or equal to 3 are considered as positive interactions.

### Data split
Dataset is split by date so that 20% of the last interactions as are placed in the test part. Cold items and users are dropped.

### Predict:
We will predict top-10 most relevant films for each user.

### Metrics
Quality metrics used:__ndcg@k, hitrate@k, map@k, mrr@k__ for k = 1, 5, 10
Additional metrics used: __coverage@k__ and __surprisal@k__.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%config Completer.use_jedi = False

In [3]:
import warnings
from optuna.exceptions import ExperimentalWarning
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=ExperimentalWarning)

In [4]:
import logging
import pandas as pd
import time


from pyspark.sql import functions as sf, types as st

from replay.data_preparator import DataPreparator
from replay.experiment import Experiment
from replay.metrics import Coverage, HitRate, MRR, MAP, NDCG, Surprisal
from replay.models import (
    ALSWrap, 
    ADMMSLIM, 
    KNN,
    LightFMWrap, 
    MultVAE, 
    NeuroMF, 
    SLIM, 
    PopRec, 
    RandomRec, 
    Wilson, 
    Word2VecRec
)

from replay.models.base_rec import HybridRecommender
from replay.session_handler import State
from replay.splitters import DateSplitter
from replay.utils import get_log_info

In [5]:
spark = State().session
spark

In [6]:
logger = logging.getLogger("replay")

In [7]:
K = 10
K_list_metrics = [1, 5, 10]
BUDGET = 20
SEED = 12345

## 0. Preprocessing <a name='data-preparator'></a>

### 0.1 Data loading

In [8]:
from rs_datasets import MovieLens

data = MovieLens("1m")
data.info()

ratings


,user_id,item_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968



users


,user_id,gender,age,occupation,zip_code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117



items


,item_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


#### log preprocessing

In [9]:
# converting log of interactions to spark-dataframe format
log = DataPreparator().transform(
    data=data.ratings,
    columns_names={
        "user_id": "user_id",
        "item_id": "item_id",
        "relevance": "rating",
        "timestamp": "timestamp"
    }
)
print(get_log_info(log))

total lines: 1000209, total users: 6040, total items: 3706


In [10]:
# will consider ratings >= 3 as positive feedback. A positive feedback is treated with relevance = 1
only_positives_log = log.filter(sf.col('relevance') >= 3).withColumn('relevance', sf.lit(1))
only_positives_log.count()

836478

In [11]:
user_features=None
item_features=None

### 0.2. Data split

In [12]:
# train/test split 
train_spl = DateSplitter(
    test_start=0.2,
    drop_cold_items=True,
    drop_cold_users=True,
)
train, test = train_spl.split(only_positives_log)
print('train info:\n', get_log_info(train))
print('test info:\n', get_log_info(test))

train info:
 total lines: 669181, total users: 5397, total items: 3569
test info:
 total lines: 86542, total users: 1139, total items: 3279


In [13]:
# train/test split for hyperparameters selection
opt_train, opt_val = train_spl.split(train)
opt_train.count(), opt_val.count()

(535343, 24241)

In [14]:
# negative feedback will be used for Wilson models
only_negatives_log = log.filter(sf.col('relevance') < 3).withColumn('relevance', sf.lit(0.))
test_start = test.agg(sf.min('timestamp')).collect()[0][0]

# train with both positive and negative feedback
pos_neg_train=(train
              .withColumn('relevance', sf.lit(1))
              .union(only_negatives_log.filter(sf.col('timestamp') < test_start))
             )
pos_neg_train.count()

798993

In [15]:
train.show(2)

+-------+-------+---------+-------------------+
|user_id|item_id|relevance|          timestamp|
+-------+-------+---------+-------------------+
|    637|   3930|        1|2000-12-02 05:30:12|
|    637|   3932|        1|2000-12-02 05:53:52|
+-------+-------+---------+-------------------+
only showing top 2 rows



# 1. Metrics definition

In [16]:
# experiment is used for metrics calculation
e = Experiment(test, {MAP(): K, NDCG(): K, HitRate(): K_list_metrics, Coverage(train): K, Surprisal(train): K, MRR(): K})

# 2. Models training

## 2.1. Non-personalized models

In [17]:
non_personalized_models = {'Popular Recommender': [PopRec(), 'no_opt'], 
          'Random Recommender (uniform)': [RandomRec(seed=SEED, distribution='uniform'), 'no_opt'], 
          'Random Recommender (popularity-based)': [RandomRec(seed=SEED, distribution='popular_based'), {"alpha": [-0.5, 100]}],
          'Wilson Recommender': [Wilson(), 'no_opt']}

In [18]:
def fit_predict_add_res(name, model, experiment, train, suffix=''):
    """
    Run fit_predict for the `model`, measure time on fit_predict and evaluate metrics
    """
    start_time=time.time()
    
    fit_predict_params = {'log': train, 'k': K, 'users': test.select('user_id').distinct()}
    if isinstance(model, Wilson):
        fit_predict_params['log'] = pos_neg_train

    if isinstance(model, HybridRecommender):
        fit_predict_params['item_features'] = item_features
        fit_predict_params['user_features'] = user_features
    
    pred=model.fit_predict(**fit_predict_params)
    pred.count()
    fit_predict_time = time.time() - start_time
    
    experiment.add_result(name + suffix, pred)
    experiment.results.loc[name + suffix, 'fit_pred_time'] = fit_predict_time
    
    print(experiment.results[['NDCG@{}'.format(K), 'MRR@{}'.format(K), 'Coverage@{}'.format(K), 'fit_pred_time']].sort_values('NDCG@{}'.format(K), ascending=False))

In [19]:
def full_pipeline(models, experiment, train, suffix='', budget=BUDGET):
    """
    For each model:
        -  if required: run hyperparameters search, set best params and save param values to `experiment`
        - pass model to `fit_predict_add_res`        
    """
    
    for name, [model, params] in models.items():
        model.logger.info(msg='{} started'.format(name))
        if params != 'no_opt':
            model.logger.info(msg='{} optimization started'.format(name))
            best_params = model.optimize(opt_train, 
                                         opt_val, 
                                         param_borders=params, 
                                         item_features=item_features,
                                         user_features=user_features,
                                         k=K, 
                                         budget=budget)
            model.set_params(**best_params)
            logger.info(msg='best params for {} are: {}'.format(name, best_params))
            experiment.results.loc[name + suffix, 'params'] = best_params.__repr__()
        
        logger.info(msg='{} fit_predict started'.format(name))
        fit_predict_add_res(name, model, experiment, train, suffix)        

In [20]:
%%time
full_pipeline(non_personalized_models, e, train)

10-Nov-21 21:18:38, replay, INFO: Popular Recommender started
10-Nov-21 21:18:38, replay, INFO: Popular Recommender fit_predict started
10-Nov-21 21:19:43, replay, INFO: Random Recommender (uniform) started
10-Nov-21 21:19:43, replay, INFO: Random Recommender (uniform) fit_predict started


                      NDCG@10    MRR@10  Coverage@10  fit_pred_time
Popular Recommender  0.243655  0.390426     0.033903      31.643974


10-Nov-21 21:20:34, replay, INFO: Random Recommender (popularity-based) started
10-Nov-21 21:20:34, replay, INFO: Random Recommender (popularity-based) optimization started
[I 2021-11-10 21:20:34,858] A new study created in memory with name: no-name-6ba6d6df-1560-4972-be34-461a75b6ba38


                               NDCG@10    MRR@10  Coverage@10  fit_pred_time
Popular Recommender           0.243655  0.390426     0.033903      31.643974
Random Recommender (uniform)  0.021545  0.051278     0.959092      15.109731


[I 2021-11-10 21:20:49,047] Trial 0 finished with value: 0.06959440322041978 and parameters: {'alpha': 0.0}. Best is trial 0 with value: 0.06959440322041978.
[I 2021-11-10 21:21:01,142] Trial 1 finished with value: 0.05868989852199487 and parameters: {'alpha': 46.846529691140894}. Best is trial 0 with value: 0.06959440322041978.
[I 2021-11-10 21:21:16,423] Trial 2 finished with value: 0.0656915125208821 and parameters: {'alpha': 4.203851476032749}. Best is trial 0 with value: 0.06959440322041978.
[I 2021-11-10 21:21:39,117] Trial 3 finished with value: 0.05863076066031065 and parameters: {'alpha': 45.535646965281074}. Best is trial 0 with value: 0.06959440322041978.
[I 2021-11-10 21:21:57,231] Trial 4 finished with value: 0.06015075426552067 and parameters: {'alpha': 69.22832139828697}. Best is trial 0 with value: 0.06959440322041978.
[I 2021-11-10 21:22:10,032] Trial 5 finished with value: 0.054783383629299076 and parameters: {'alpha': 86.78363170265781}. Best is trial 0 with value: 0

                                        NDCG@10    MRR@10  Coverage@10  \
Popular Recommender                    0.243655  0.390426     0.033903   
Random Recommender (popularity-based)  0.066275  0.147458     0.772205   
Random Recommender (uniform)           0.021545  0.051278     0.959092   

                                       fit_pred_time  
Popular Recommender                        31.643974  
Random Recommender (popularity-based)      13.282000  
Random Recommender (uniform)               15.109731  
                                        NDCG@10    MRR@10  Coverage@10  \
Popular Recommender                    0.243655  0.390426     0.033903   
Wilson Recommender                     0.092121  0.180976     0.017092   
Random Recommender (popularity-based)  0.066275  0.147458     0.772205   
Random Recommender (uniform)           0.021545  0.051278     0.959092   

                                       fit_pred_time  
Popular Recommender                        31.643974  
Wi

In [21]:
e.results.sort_values('NDCG@10', ascending=False)

,Coverage@10,HitRate@1,HitRate@5,HitRate@10,MAP@10,MRR@10,NDCG@10,Surprisal@10,fit_pred_time,params
Popular Recommender,0.033903,0.284460,0.530290,0.645303,0.157239,0.390426,0.243655,0.118354,31.643974,NaN
Wilson Recommender,0.017092,0.083406,0.345040,0.414399,0.045002,0.180976,0.092121,0.262190,24.320912,NaN
Random Recommender (popularity-based),0.772205,0.072871,0.236172,0.371378,0.027830,0.147458,0.066275,0.348670,13.282000,{'alpha': 26.93797623564389}
Random Recommender (uniform),0.959092,0.018437,0.093064,0.159789,0.007946,0.051278,0.021545,0.538118,15.109731,NaN


In [22]:
e.results.to_csv('res_21_rel_1.csv')

## 2.2  Personalized models without features

In [23]:
common_models = {
          'ADMM SLIM': [ADMMSLIM(seed=SEED), {"lambda_1": [1e-6, 10],
                                              "lambda_2": [1e-6, 1000]},],
          'Implicit ALS': [ALSWrap(seed=SEED), None], 
          'Explicit ALS': [ALSWrap(seed=SEED, implicit_prefs=False), None], 
          'KNN': [KNN(), None], 
          'LightFM': [LightFMWrap(random_state=SEED), {"no_components": [8, 512]}], 
          'SLIM': [SLIM(seed=SEED), None]}

In [24]:
%%time
full_pipeline(common_models, e, train)

10-Nov-21 21:26:38, replay, INFO: ADMM SLIM started
10-Nov-21 21:26:38, replay, INFO: ADMM SLIM optimization started
[I 2021-11-10 21:26:38,164] A new study created in memory with name: no-name-b50c5fbc-f4f6-4153-9797-09be56714e2f
[I 2021-11-10 21:27:11,361] Trial 0 finished with value: 0.21171788539708894 and parameters: {'lambda_1': 0.05376934338684738, 'lambda_2': 1.0428581841682532}. Best is trial 0 with value: 0.21171788539708894.
[I 2021-11-10 21:27:32,695] Trial 1 finished with value: 0.16269986608211737 and parameters: {'lambda_1': 0.00010906587507318183, 'lambda_2': 0.15351162190197262}. Best is trial 0 with value: 0.21171788539708894.
[I 2021-11-10 21:27:51,988] Trial 2 finished with value: 0.1626139646313865 and parameters: {'lambda_1': 4.2345238618946585e-05, 'lambda_2': 9.513948052478778}. Best is trial 0 with value: 0.21171788539708894.
[I 2021-11-10 21:28:15,012] Trial 3 finished with value: 0.13908296009528973 and parameters: {'lambda_1': 0.0003912333150524383, 'lambda_

                                        NDCG@10    MRR@10  Coverage@10  \
Popular Recommender                    0.243655  0.390426     0.033903   
ADMM SLIM                              0.150436  0.282925     0.537686   
Wilson Recommender                     0.092121  0.180976     0.017092   
Random Recommender (popularity-based)  0.066275  0.147458     0.772205   
Random Recommender (uniform)           0.021545  0.051278     0.959092   

                                       fit_pred_time  
Popular Recommender                        31.643974  
ADMM SLIM                                  39.125110  
Wilson Recommender                         24.320912  
Random Recommender (popularity-based)      13.282000  
Random Recommender (uniform)               15.109731  


/home/volodkevich/replay_tasks/replay_performance/opt_env/lib/python3.7/site-packages/optuna/distributions.py:364: FutureWarning: Samplers and other components in Optuna will assume that `step` is 1. `step` argument is deprecated and will be removed in the future. The removal of this feature is currently scheduled for v4.0.0, but this schedule is subject to change.
  FutureWarning,
[I 2021-11-10 21:37:28,549] Trial 0 finished with value: 0.20886466861222316 and parameters: {'rank': 10}. Best is trial 0 with value: 0.20886466861222316.
[I 2021-11-10 21:37:44,503] Trial 1 finished with value: 0.18165649922762667 and parameters: {'rank': 51}. Best is trial 0 with value: 0.20886466861222316.
[I 2021-11-10 21:38:16,956] Trial 2 finished with value: 0.17166181242469755 and parameters: {'rank': 140}. Best is trial 0 with value: 0.20886466861222316.
[I 2021-11-10 21:38:29,024] Trial 3 finished with value: 0.21106983618065447 and parameters: {'rank': 8}. Best is trial 3 with value: 0.2110698361

                                        NDCG@10    MRR@10  Coverage@10  \
Implicit ALS                           0.265372  0.419297     0.132810   
Popular Recommender                    0.243655  0.390426     0.033903   
ADMM SLIM                              0.150436  0.282925     0.537686   
Wilson Recommender                     0.092121  0.180976     0.017092   
Random Recommender (popularity-based)  0.066275  0.147458     0.772205   
Random Recommender (uniform)           0.021545  0.051278     0.959092   

                                       fit_pred_time  
Implicit ALS                               16.260996  
Popular Recommender                        31.643974  
ADMM SLIM                                  39.125110  
Wilson Recommender                         24.320912  
Random Recommender (popularity-based)      13.282000  
Random Recommender (uniform)               15.109731  


/home/volodkevich/replay_tasks/replay_performance/opt_env/lib/python3.7/site-packages/optuna/distributions.py:364: FutureWarning: Samplers and other components in Optuna will assume that `step` is 1. `step` argument is deprecated and will be removed in the future. The removal of this feature is currently scheduled for v4.0.0, but this schedule is subject to change.
  FutureWarning,
[I 2021-11-10 21:44:11,026] Trial 0 finished with value: 0.021652418765322076 and parameters: {'rank': 10}. Best is trial 0 with value: 0.021652418765322076.
[I 2021-11-10 21:44:25,016] Trial 1 finished with value: 0.020481860022303955 and parameters: {'rank': 17}. Best is trial 0 with value: 0.021652418765322076.
[I 2021-11-10 21:44:39,138] Trial 2 finished with value: 0.01946704345238265 and parameters: {'rank': 56}. Best is trial 0 with value: 0.021652418765322076.
[I 2021-11-10 21:44:50,607] Trial 3 finished with value: 0.018017752054901526 and parameters: {'rank': 9}. Best is trial 0 with value: 0.02165

...

                                        NDCG@10    MRR@10  Coverage@10  \
Implicit ALS                           0.265372  0.419297     0.132810   
KNN                                    0.255116  0.408756     0.058840   
Popular Recommender                    0.243655  0.390426     0.033903   
ADMM SLIM                              0.150436  0.282925     0.537686   
Wilson Recommender                     0.092121  0.180976     0.017092   
Random Recommender (popularity-based)  0.066275  0.147458     0.772205   
Explicit ALS                           0.032465  0.078856     0.540768   
Random Recommender (uniform)           0.021545  0.051278     0.959092   

                                       fit_pred_time  
Implicit ALS                               16.260996  
KNN                                        23.151356  
Popular Recommender                        31.643974  
ADMM SLIM                                  39.125110  
Wilson Recommender                         24.320912  
Ran

/home/volodkevich/replay_tasks/replay_performance/opt_env/lib/python3.7/site-packages/optuna/distributions.py:364: FutureWarning: Samplers and other components in Optuna will assume that `step` is 1. `step` argument is deprecated and will be removed in the future. The removal of this feature is currently scheduled for v4.0.0, but this schedule is subject to change.
  FutureWarning,
[I 2021-11-10 21:58:47,572] Trial 0 finished with value: 0.18572548043418846 and parameters: {'no_components': 128}. Best is trial 0 with value: 0.18572548043418846.
[I 2021-11-10 21:58:58,464] Trial 1 finished with value: 0.19945917721349576 and parameters: {'no_components': 41}. Best is trial 1 with value: 0.19945917721349576.
[I 2021-11-10 21:59:08,895] Trial 2 finished with value: 0.21558616880328763 and parameters: {'no_components': 16}. Best is trial 2 with value: 0.21558616880328763.
[I 2021-11-10 21:59:19,209] Trial 3 finished with value: 0.21508670315330183 and parameters: {'no_components': 10}. Bes

                                        NDCG@10    MRR@10  Coverage@10  \
Implicit ALS                           0.265372  0.419297     0.132810   
LightFM                                0.264699  0.426189     0.149902   
KNN                                    0.255116  0.408756     0.058840   
Popular Recommender                    0.243655  0.390426     0.033903   
ADMM SLIM                              0.150436  0.282925     0.537686   
Wilson Recommender                     0.092121  0.180976     0.017092   
Random Recommender (popularity-based)  0.066275  0.147458     0.772205   
Explicit ALS                           0.032465  0.078856     0.540768   
Random Recommender (uniform)           0.021545  0.051278     0.959092   

                                       fit_pred_time  
Implicit ALS                               16.260996  
LightFM                                    13.635783  
KNN                                        23.151356  
Popular Recommender                    

[I 2021-11-10 22:03:03,147] Trial 0 finished with value: 0.22679042140897618 and parameters: {'beta': 4.0, 'lambda_': 0.02}. Best is trial 0 with value: 0.22679042140897618.
[I 2021-11-10 22:03:14,337] Trial 1 finished with value: 0.2117927681013085 and parameters: {'beta': 0.11602050205463056, 'lambda_': 0.141837047839974}. Best is trial 0 with value: 0.22679042140897618.
[I 2021-11-10 22:03:30,502] Trial 2 finished with value: 0.17632753025551723 and parameters: {'beta': 4.289991608690477e-05, 'lambda_': 0.0001907858025072501}. Best is trial 0 with value: 0.22679042140897618.
[I 2021-11-10 22:03:47,268] Trial 3 finished with value: 0.1770002453818469 and parameters: {'beta': 0.000161806798148545, 'lambda_': 0.00010962760388659542}. Best is trial 0 with value: 0.22679042140897618.
[I 2021-11-10 22:03:57,361] Trial 4 finished with value: 0.0 and parameters: {'beta': 0.3263924992373147, 'lambda_': 0.3416737913086433}. Best is trial 0 with value: 0.22679042140897618.
[I 2021-11-10 22:04:

                                        NDCG@10    MRR@10  Coverage@10  \
SLIM                                   0.269347  0.434673     0.060521   
Implicit ALS                           0.265372  0.419297     0.132810   
LightFM                                0.264699  0.426189     0.149902   
KNN                                    0.255116  0.408756     0.058840   
Popular Recommender                    0.243655  0.390426     0.033903   
ADMM SLIM                              0.150436  0.282925     0.537686   
Wilson Recommender                     0.092121  0.180976     0.017092   
Random Recommender (popularity-based)  0.066275  0.147458     0.772205   
Explicit ALS                           0.032465  0.078856     0.540768   
Random Recommender (uniform)           0.021545  0.051278     0.959092   

                                       fit_pred_time  
SLIM                                       23.937692  
Implicit ALS                               16.260996  
LightFM             

In [25]:
e.results.sort_values('NDCG@10', ascending=False)

,Coverage@10,HitRate@1,HitRate@5,HitRate@10,MAP@10,MRR@10,NDCG@10,Surprisal@10,fit_pred_time,params
SLIM,0.060521,0.326602,0.582968,0.690957,0.175038,0.434673,0.269347,0.133177,23.937692,"{'beta': 4.593211196648865, 'lambda_': 0.00525..."
Implicit ALS,0.132810,0.305531,0.569798,0.685689,0.171672,0.419297,0.265372,0.162866,16.260996,{'rank': 8}
LightFM,0.149902,0.303775,0.589113,0.693591,0.169451,0.426189,0.264699,0.165916,13.635783,{'no_components': 8}
KNN,0.058840,0.299385,0.557507,0.647059,0.165913,0.408756,0.255116,0.142944,23.151356,"{'num_neighbours': 99, 'shrink': 33}"
Popular Recommender,0.033903,0.284460,0.530290,0.645303,0.157239,0.390426,0.243655,0.118354,31.643974,NaN
ADMM SLIM,0.537686,0.179104,0.417032,0.534680,0.082699,0.282925,0.150436,0.312423,39.125110,"{'lambda_1': 0.05376934338684738, 'lambda_2': ..."
Wilson Recommender,0.017092,0.083406,0.345040,0.414399,0.045002,0.180976,0.092121,0.262190,24.320912,NaN
Random Recommender (popularity-based),0.772205,0.072871,0.236172,0.371378,0.027830,0.147458,0.066275,0.348670,13.282000,{'alpha': 26.93797623564389}
Explicit ALS,0.540768,0.031607,0.135206,0.213345,0.012413,0.078856,0.032465,0.527453,25.089717,{'rank': 77}
Random Recommender (uniform),0.959092,0.018437,0.093064,0.159789,0.007946,0.051278,0.021545,0.538118,15.109731,NaN


In [26]:
e.results.to_csv('res_22_rel_1.csv')

## 2.3 Neural models

In [27]:
nets = {'MultVAE with default parameters': [MultVAE(), 'no_opt'],
        'NeuroMF with default parameters': [NeuroMF(), 'no_opt'], 
        'Word2Vec with default parameters': [Word2VecRec(seed=SEED), 'no_opt'],
        'MultVAE with optimized parameters': [MultVAE(), {"learning_rate": [0.001, 0.5],
                                   "dropout": [0, 0.5],
                                    "l2_reg": [1e-6, 5]
                                   }],
        'NeuroMF with optimized parameters': [NeuroMF(), {
                                    "learning_rate": [0.001, 0.5],
                                    "l2_reg": [1e-6, 5],
                                    "count_negative_sample": [1, 20]
                                    }],
        'Word2Vec with optimized parameters': [Word2VecRec(seed=SEED), None]}

10-Nov-21 22:08:43, replay, INFO: The model is neural network with non-distributed training
10-Nov-21 22:08:43, replay, INFO: The model is neural network with non-distributed training
10-Nov-21 22:08:43, replay, INFO: The model is neural network with non-distributed training
10-Nov-21 22:08:43, replay, INFO: The model is neural network with non-distributed training


In [28]:
%%time
full_pipeline(nets, e, train, budget=10)

10-Nov-21 22:08:43, replay, INFO: MultVAE with default parameters started
10-Nov-21 22:08:43, replay, INFO: MultVAE with default parameters fit_predict started
2021-11-10 22:08:58,838 ignite.handlers.early_stopping.EarlyStopping INFO: EarlyStopping: Stop training
10-Nov-21 22:10:55, replay, INFO: NeuroMF with default parameters started
10-Nov-21 22:10:55, replay, INFO: NeuroMF with default parameters fit_predict started


                                        NDCG@10    MRR@10  Coverage@10  \
SLIM                                   0.269347  0.434673     0.060521   
Implicit ALS                           0.265372  0.419297     0.132810   
LightFM                                0.264699  0.426189     0.149902   
KNN                                    0.255116  0.408756     0.058840   
Popular Recommender                    0.243655  0.390426     0.033903   
MultVAE with default parameters        0.235500  0.380018     0.033343   
ADMM SLIM                              0.150436  0.282925     0.537686   
Wilson Recommender                     0.092121  0.180976     0.017092   
Random Recommender (popularity-based)  0.066275  0.147458     0.772205   
Explicit ALS                           0.032465  0.078856     0.540768   
Random Recommender (uniform)           0.021545  0.051278     0.959092   

                                       fit_pred_time  
SLIM                                       23.937692  
I

2021-11-10 22:15:08,393 ignite.handlers.early_stopping.EarlyStopping INFO: EarlyStopping: Stop training
10-Nov-21 22:21:16, replay, INFO: Word2Vec with default parameters started
10-Nov-21 22:21:16, replay, INFO: Word2Vec with default parameters fit_predict started


                                        NDCG@10    MRR@10  Coverage@10  \
SLIM                                   0.269347  0.434673     0.060521   
Implicit ALS                           0.265372  0.419297     0.132810   
LightFM                                0.264699  0.426189     0.149902   
KNN                                    0.255116  0.408756     0.058840   
Popular Recommender                    0.243655  0.390426     0.033903   
MultVAE with default parameters        0.235500  0.380018     0.033343   
NeuroMF with default parameters        0.205890  0.345424     0.240123   
ADMM SLIM                              0.150436  0.282925     0.537686   
Wilson Recommender                     0.092121  0.180976     0.017092   
Random Recommender (popularity-based)  0.066275  0.147458     0.772205   
Explicit ALS                           0.032465  0.078856     0.540768   
Random Recommender (uniform)           0.021545  0.051278     0.959092   

                                     

10-Nov-21 22:22:52, replay, INFO: MultVAE with optimized parameters started
10-Nov-21 22:22:52, replay, INFO: MultVAE with optimized parameters optimization started
[I 2021-11-10 22:22:52,814] A new study created in memory with name: no-name-d58b0bea-1ad6-4c35-9ed0-5605c4022ff3


...

                                        NDCG@10    MRR@10  Coverage@10  \
SLIM                                   0.269347  0.434673     0.060521   
Implicit ALS                           0.265372  0.419297     0.132810   
LightFM                                0.264699  0.426189     0.149902   
KNN                                    0.255116  0.408756     0.058840   
Popular Recommender                    0.243655  0.390426     0.033903   
MultVAE with optimized parameters      0.235931  0.387634     0.029700   
MultVAE with default parameters        0.235500  0.380018     0.033343   
NeuroMF with default parameters        0.205890  0.345424     0.240123   
ADMM SLIM                              0.150436  0.282925     0.537686   
Word2Vec with default parameters       0.136424  0.242314     0.138975   
Wilson Recommender                     0.092121  0.180976     0.017092   
Random Recommender (popularity-based)  0.066275  0.147458     0.772205   
Explicit ALS                          

2021-11-10 22:34:50,845 ignite.handlers.early_stopping.EarlyStopping INFO: EarlyStopping: Stop training
[I 2021-11-10 22:35:20,112] Trial 0 finished with value: 0.14861627613491488 and parameters: {'learning_rate': 0.1157186326942789, 'l2_reg': 0.0007093754069867472, 'count_negative_sample': 3}. Best is trial 0 with value: 0.14861627613491488.
[I 2021-11-10 22:50:20,561] Trial 1 finished with value: 0.2169293855232518 and parameters: {'learning_rate': 0.0017481102870074488, 'l2_reg': 0.0011088335980149711, 'count_negative_sample': 6}. Best is trial 1 with value: 0.2169293855232518.
[I 2021-11-10 23:25:46,058] Trial 2 finished with value: 0.010850934776615093 and parameters: {'learning_rate': 0.3199148733307085, 'l2_reg': 0.0020216102123139686, 'count_negative_sample': 19}. Best is trial 1 with value: 0.2169293855232518.
2021-11-10 23:56:57,714 ignite.handlers.early_stopping.EarlyStopping INFO: EarlyStopping: Stop training
[I 2021-11-10 23:57:28,421] Trial 3 finished with value: 0.16803

                                        NDCG@10    MRR@10  Coverage@10  \
SLIM                                   0.269347  0.434673     0.060521   
Implicit ALS                           0.265372  0.419297     0.132810   
LightFM                                0.264699  0.426189     0.149902   
KNN                                    0.255116  0.408756     0.058840   
Popular Recommender                    0.243655  0.390426     0.033903   
MultVAE with optimized parameters      0.235931  0.387634     0.029700   
MultVAE with default parameters        0.235500  0.380018     0.033343   
NeuroMF with optimized parameters      0.208904  0.301676     0.034744   
NeuroMF with default parameters        0.205890  0.345424     0.240123   
ADMM SLIM                              0.150436  0.282925     0.537686   
Word2Vec with default parameters       0.136424  0.242314     0.138975   
Wilson Recommender                     0.092121  0.180976     0.017092   
Random Recommender (popularity-based) 

[I 2021-11-11 02:12:08,586] Trial 0 finished with value: 0.13403044530305955 and parameters: {'rank': 100, 'window_size': 1, 'use_idf': False}. Best is trial 0 with value: 0.13403044530305955.
[I 2021-11-11 02:16:48,529] Trial 1 finished with value: 0.03668316957766785 and parameters: {'rank': 246, 'window_size': 74, 'use_idf': True}. Best is trial 0 with value: 0.13403044530305955.
[I 2021-11-11 02:18:45,232] Trial 2 finished with value: 0.04059492320787539 and parameters: {'rank': 53, 'window_size': 100, 'use_idf': False}. Best is trial 0 with value: 0.13403044530305955.
[I 2021-11-11 02:21:37,679] Trial 3 finished with value: 0.042521323164824876 and parameters: {'rank': 224, 'window_size': 41, 'use_idf': False}. Best is trial 0 with value: 0.13403044530305955.
[I 2021-11-11 02:23:05,596] Trial 4 finished with value: 0.034942189355504014 and parameters: {'rank': 64, 'window_size': 55, 'use_idf': True}. Best is trial 0 with value: 0.13403044530305955.
[I 2021-11-11 02:25:27,394] Tria

                                        NDCG@10    MRR@10  Coverage@10  \
SLIM                                   0.269347  0.434673     0.060521   
Implicit ALS                           0.265372  0.419297     0.132810   
LightFM                                0.264699  0.426189     0.149902   
KNN                                    0.255116  0.408756     0.058840   
Popular Recommender                    0.243655  0.390426     0.033903   
MultVAE with optimized parameters      0.235931  0.387634     0.029700   
MultVAE with default parameters        0.235500  0.380018     0.033343   
NeuroMF with optimized parameters      0.208904  0.301676     0.034744   
NeuroMF with default parameters        0.205890  0.345424     0.240123   
ADMM SLIM                              0.150436  0.282925     0.537686   
Word2Vec with default parameters       0.136424  0.242314     0.138975   
Word2Vec with optimized parameters     0.136424  0.242314     0.138975   
Wilson Recommender                    

In [29]:
e.results.sort_values('NDCG@10', ascending=False)

,Coverage@10,HitRate@1,HitRate@5,HitRate@10,MAP@10,MRR@10,NDCG@10,Surprisal@10,fit_pred_time,params
SLIM,0.060521,0.326602,0.582968,0.690957,0.175038,0.434673,0.269347,0.133177,23.937692,"{'beta': 4.593211196648865, 'lambda_': 0.00525..."
Implicit ALS,0.132810,0.305531,0.569798,0.685689,0.171672,0.419297,0.265372,0.162866,16.260996,{'rank': 8}
LightFM,0.149902,0.303775,0.589113,0.693591,0.169451,0.426189,0.264699,0.165916,13.635783,{'no_components': 8}
KNN,0.058840,0.299385,0.557507,0.647059,0.165913,0.408756,0.255116,0.142944,23.151356,"{'num_neighbours': 99, 'shrink': 33}"
Popular Recommender,0.033903,0.284460,0.530290,0.645303,0.157239,0.390426,0.243655,0.118354,31.643974,NaN
MultVAE with optimized parameters,0.029700,0.283582,0.524144,0.640035,0.150055,0.387634,0.235931,0.123279,35.335575,"{'learning_rate': 0.44918917641176687, 'dropou..."
MultVAE with default parameters,0.033343,0.262511,0.535558,0.643547,0.149160,0.380018,0.235500,0.121732,44.113874,NaN
NeuroMF with optimized parameters,0.034744,0.138718,0.526778,0.640913,0.124058,0.301676,0.208904,0.198387,1149.123512,"{'learning_rate': 0.0017481102870074488, 'l2_r..."
NeuroMF with default parameters,0.240123,0.215101,0.516242,0.644425,0.117832,0.345424,0.205890,0.231599,328.956183,NaN
ADMM SLIM,0.537686,0.179104,0.417032,0.534680,0.082699,0.282925,0.150436,0.312423,39.125110,"{'lambda_1': 0.05376934338684738, 'lambda_2': ..."


In [30]:
e.results.to_csv('res_23_rel_1.csv')

## 2.4 Models considering features

### 2.4.1 item features preprocessing

In [31]:
%%time
item_features = DataPreparator().transform(
    data=data.items,
    columns_names={
        "item_id": "item_id"
    }
)

CPU times: user 29.3 ms, sys: 13.8 ms, total: 43.1 ms
Wall time: 134 ms


In [32]:
item_features.show(2)

+-------+--------------------+----------------+
|item_id|              genres|           title|
+-------+--------------------+----------------+
|      1|Animation|Childre...|Toy Story (1995)|
|      2|Adventure|Childre...|  Jumanji (1995)|
+-------+--------------------+----------------+
only showing top 2 rows



In [33]:
year = item_features.withColumn('year', sf.substring(sf.col('title'), -5, 4).astype(st.IntegerType())).select('item_id', 'year')
year.show(2)

+-------+----+
|item_id|year|
+-------+----+
|      1|1995|
|      2|1995|
+-------+----+
only showing top 2 rows



In [34]:
genres = (
    State().session.createDataFrame(data.items[["item_id", "genres"]])
    .select(
        "item_id",
        sf.split("genres", "\|").alias("genres")
    )
)

In [35]:
genres_list = (
    genres.select(sf.explode("genres").alias("genre"))
    .distinct().filter('genre <> "(no genres listed)"')
    .toPandas()["genre"].tolist()
)

In [36]:
genres_list

['Documentary',
 'Adventure',
 'Animation',
 'Comedy',
 'Thriller',
 'Sci-Fi',
 'Musical',
 'Horror',
 'Action',
 'Fantasy',
 'War',
 'Mystery',
 "Children's",
 'Drama',
 'Film-Noir',
 'Crime',
 'Western',
 'Romance']

In [37]:
from pyspark.sql.functions import col, array_contains
from pyspark.sql.types import IntegerType

item_features = genres
for genre in genres_list:
    item_features = item_features.withColumn(
        genre,
        array_contains(col("genres"), genre).astype(IntegerType())
    )
item_features = item_features.drop("genres").cache()
item_features.count()

3883

In [38]:
item_features = item_features.join(year, on='item_id', how='inner')
item_features.count()

3883

In [39]:
item_features.cache()

DataFrame[item_id: int, Documentary: int, Adventure: int, Animation: int, Comedy: int, Thriller: int, Sci-Fi: int, Musical: int, Horror: int, Action: int, Fantasy: int, War: int, Mystery: int, Children's: int, Drama: int, Film-Noir: int, Crime: int, Western: int, Romance: int, year: int]

In [40]:
item_features.show(3)

+-------+-----------+---------+---------+------+--------+------+-------+------+------+-------+---+-------+----------+-----+---------+-----+-------+-------+----+
|item_id|Documentary|Adventure|Animation|Comedy|Thriller|Sci-Fi|Musical|Horror|Action|Fantasy|War|Mystery|Children's|Drama|Film-Noir|Crime|Western|Romance|year|
+-------+-----------+---------+---------+------+--------+------+-------+------+------+-------+---+-------+----------+-----+---------+-----+-------+-------+----+
|      1|          0|        0|        1|     1|       0|     0|      0|     0|     0|      0|  0|      0|         1|    0|        0|    0|      0|      0|1995|
|      2|          0|        1|        0|     0|       0|     0|      0|     0|     0|      1|  0|      0|         1|    0|        0|    0|      0|      0|1995|
|      3|          0|        0|        0|     1|       0|     0|      0|     0|     0|      0|  0|      0|         0|    0|        0|    0|      0|      1|1995|
+-------+-----------+---------+---

### 2.4.2 Models training

In [41]:
models_with_features = {'LightFM with item features': [LightFMWrap(random_state=SEED), {"no_components": [8, 512]}]}

In [42]:
%%time
full_pipeline(models_with_features, e, train)

11-Nov-21 02:42:22, replay, INFO: LightFM with item features started
11-Nov-21 02:42:22, replay, INFO: LightFM with item features optimization started
[I 2021-11-11 02:42:22,233] A new study created in memory with name: no-name-83bd088c-4d2f-4203-8638-c26d5798434e
/home/volodkevich/replay_tasks/replay_performance/opt_env/lib/python3.7/site-packages/optuna/distributions.py:364: FutureWarning: Samplers and other components in Optuna will assume that `step` is 1. `step` argument is deprecated and will be removed in the future. The removal of this feature is currently scheduled for v4.0.0, but this schedule is subject to change.
  FutureWarning,
[I 2021-11-11 02:43:54,703] Trial 0 finished with value: 0.1899535560848137 and parameters: {'no_components': 128}. Best is trial 0 with value: 0.1899535560848137.
[I 2021-11-11 02:44:31,736] Trial 1 finished with value: 0.1865022237319913 and parameters: {'no_components': 9}. Best is trial 0 with value: 0.1899535560848137.
[I 2021-11-11 02:45:52,4

                                        NDCG@10    MRR@10  Coverage@10  \
SLIM                                   0.269347  0.434673     0.060521   
Implicit ALS                           0.265372  0.419297     0.132810   
LightFM                                0.264699  0.426189     0.149902   
KNN                                    0.255116  0.408756     0.058840   
LightFM with item features             0.247812  0.409083     0.255534   
Popular Recommender                    0.243655  0.390426     0.033903   
MultVAE with optimized parameters      0.235931  0.387634     0.029700   
MultVAE with default parameters        0.235500  0.380018     0.033343   
NeuroMF with optimized parameters      0.208904  0.301676     0.034744   
NeuroMF with default parameters        0.205890  0.345424     0.240123   
ADMM SLIM                              0.150436  0.282925     0.537686   
Word2Vec with default parameters       0.136424  0.242314     0.138975   
Word2Vec with optimized parameters    

In [43]:
e.results.sort_values('NDCG@10', ascending=False)

,Coverage@10,HitRate@1,HitRate@5,HitRate@10,MAP@10,MRR@10,NDCG@10,Surprisal@10,fit_pred_time,params
SLIM,0.060521,0.326602,0.582968,0.690957,0.175038,0.434673,0.269347,0.133177,23.937692,"{'beta': 4.593211196648865, 'lambda_': 0.00525..."
Implicit ALS,0.132810,0.305531,0.569798,0.685689,0.171672,0.419297,0.265372,0.162866,16.260996,{'rank': 8}
LightFM,0.149902,0.303775,0.589113,0.693591,0.169451,0.426189,0.264699,0.165916,13.635783,{'no_components': 8}
KNN,0.058840,0.299385,0.557507,0.647059,0.165913,0.408756,0.255116,0.142944,23.151356,"{'num_neighbours': 99, 'shrink': 33}"
LightFM with item features,0.255534,0.284460,0.582090,0.697981,0.152378,0.409083,0.247812,0.200273,117.104723,{'no_components': 165}
Popular Recommender,0.033903,0.284460,0.530290,0.645303,0.157239,0.390426,0.243655,0.118354,31.643974,NaN
MultVAE with optimized parameters,0.029700,0.283582,0.524144,0.640035,0.150055,0.387634,0.235931,0.123279,35.335575,"{'learning_rate': 0.44918917641176687, 'dropou..."
MultVAE with default parameters,0.033343,0.262511,0.535558,0.643547,0.149160,0.380018,0.235500,0.121732,44.113874,NaN
NeuroMF with optimized parameters,0.034744,0.138718,0.526778,0.640913,0.124058,0.301676,0.208904,0.198387,1149.123512,"{'learning_rate': 0.0017481102870074488, 'l2_r..."
NeuroMF with default parameters,0.240123,0.215101,0.516242,0.644425,0.117832,0.345424,0.205890,0.231599,328.956183,NaN


In [44]:
e.results.to_csv('res_25_rel_1.csv')

In [24]:
df = e.results.drop([
    'Stack Recommender (LightFM + KNN + ALS)', 
    'NeuroMF with optimized parameters', 
    'MultVAE with default parameters', 
    'Classifier Recommender',
    'Word2Vec with optimized parameters'
]).rename(
    index={
        'Popular Recommender': 'PopRec', 
        'Random Recommender (uniform)': 'RandomRec (uniform)', 
        'Random Recommender (popularity-based)': 'RandomRec (popular)',
        'Wilson Recommender': 'Wilson', 'Implicit ALS': 'ALS (Implicit)', 'Explicit ALS': 'ALS (Explicit)',
        'NeuroMF with default parameters': 'NeuroMF', 'MultVAE with optimized parameters': 'MultVAE',
        'Word2Vec with default parameters': 'Word2Vec', 'LightFM with item features': 'LightFM (w/ feats)'
                }).sort_values('NDCG@10', ascending=False)
df

,Coverage@10,HitRate@1,HitRate@5,HitRate@10,MAP@10,MRR@10,NDCG@10,Surprisal@10,fit_pred_time,params
ALS (Implicit),0.132810,0.305531,0.569798,0.685689,0.171672,0.419297,0.265372,0.162866,10.127573,{'rank': 8}
LightFM,0.151303,0.317823,0.574188,0.698859,0.167327,0.431049,0.262777,0.168066,10.833092,{'no_components': 8}
SLIM,0.040347,0.310799,0.567164,0.669008,0.171509,0.418741,0.261370,0.123728,12.456171,"{'beta': 4.528603379741062, 'lambda_': 0.01886..."
KNN,0.055758,0.294996,0.555751,0.654960,0.166407,0.408699,0.256174,0.137584,17.963558,"{'num_neighbours': 56, 'shrink': 99}"
LightFM (item features),0.231718,0.287972,0.585601,0.690957,0.159831,0.412271,0.254673,0.194597,86.704299,{'no_components': 78}
PopRec,0.033903,0.284460,0.530290,0.645303,0.157194,0.390414,0.243614,0.118354,12.300273,NaN
MultVAE,0.030821,0.287094,0.543459,0.640035,0.150969,0.395733,0.237955,0.122875,26.977435,"{'learning_rate': 0.010693178531368242, 'dropo..."
NeuroMF,0.257495,0.187006,0.501317,0.626866,0.110592,0.317911,0.193122,0.235454,350.737231,NaN
ADMM SLIM,0.366769,0.188762,0.460053,0.590869,0.084121,0.303578,0.159086,0.236767,77.647394,"{'lambda_1': 0.0017369838173267552, 'lambda_2'..."
Word2Vec,0.145139,0.136084,0.385426,0.515364,0.072215,0.243760,0.137660,0.239778,25.133495,NaN


In [27]:
df.index.name = 'Model'

In [22]:
df = df.round(3)[['HitRate@10', 'MAP@10', 'MRR@10', 'NDCG@10', 'Coverage@10', 'Surprisal@10', 'fit_pred_time']]
df = df.rename(columns={'HitRate@10': 'HitRate', 'MAP@10': 'MAP', 'MRR@10': 'MRR',
                        'NDCG@10': 'NDCG', 'Coverage@10': 'Coverage', 
                        'Surprisal@10': 'Surprisal'})
df.to_csv('res_1m.csv')

# 3. Results

The best results by quality and time were shown by the commonly-used models such as ALS, SLIM and LightFM. 